## 한국어 임베딩과 제미니AI를 이용한 무료 RAG 시스템 구축
출처: https://anpigon.tistory.com/460

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.llms import HuggingFaceHub

import os

### 한국어 임베디이 모델 구성

In [7]:
hf = HuggingFaceEmbeddings(model_name='jhgan/ko-sroberta-multitask')

In [ ]:
docs = [
    "오늘은 정말 바쁜 하루였다. 아침에 일찍 일어나서 친구와 만나기로 한 약속을 지켰다.",
    "최근 인공지능 기술의 발전으로 많은 산업 분야에서 혁신이 일어나고 있다. 특히 자연어 처리 기술의 진보가 눈부시다.",
    "제주도는 한국에서 가장 인기 있는 관광지 중 하나다. 아름다운 해변과 푸른 자연이 매력적인 곳이다.",
    "한국의 김치는 세계적으로 유명한 발효 음식이다. 매콤하고 시원한 맛이 특징이며 다양한 요리에 활용된다.",
    "정기적인 운동과 균형 잡힌 식단은 건강을 유지하는 데 필수적이다. 매일 조금씩이라도 몸을 움직이는 습관을 들이자.",
    "온라인 학습 플랫폼의 발전으로 어디서나 다양한 지식을 접할 수 있게 되었다. 평생교육의 중요성이 점점 더 커지고 있다.",
    "지속 가능한 생활을 위해서는 일회용품 사용을 줄이고 재활용을 적극적으로 실천해야 한다.",
    "축구는 전 세계적으로 사랑받는 스포츠 중 하나다. 팀워크와 개인 기술이 조화를 이루어야 승리할 수 있다.",
    "한국의 전통 문화 중 하나인 한복은 그 아름다움으로 많은 사람들에게 사랑받고 있다. 특별한 날에 한복을 입는 것은 큰 의미가 있다.",
    "최근 글로벌 경제는 여러 도전에 직면해 있다. 변동성이 큰 시장에서 기업과 개인은 더욱 신중한 결정을 내려야 한다."
]
 
vectorstore = FAISS.from_texts(docs, embedding=hf) # hugging face의 무료 embedding 모델을 사용용
retriever = vectorstore.as_retriever()

In [39]:
query = '매운 음식이 뭐가 있지'
retriever.get_relevant_documents(query)

[Document(id='f1e2d73e-c7ff-4848-ad6b-ea61972ee53d', metadata={}, page_content='한국의 김치는 세계적으로 유명한 발효 음식이다. 매콤하고 시원한 맛이 특징이며 다양한 요리에 활용된다.'),
 Document(id='58e7a828-68d9-47f8-8cd2-b4a8869d48ec', metadata={}, page_content='한국의 전통 문화 중 하나인 한복은 그 아름다움으로 많은 사람들에게 사랑받고 있다. 특별한 날에 한복을 입는 것은 큰 의미가 있다.'),
 Document(id='85ee3d9a-933b-4316-bbf9-6d6de2ec64c9', metadata={}, page_content='오늘은 정말 바쁜 하루였다. 아침에 일찍 일어나서 친구와 만나기로 한 약속을 지켰다.'),
 Document(id='4ea3cf99-57c9-446c-b506-9473f02df45d', metadata={}, page_content='축구는 전 세계적으로 사랑받는 스포츠 중 하나다. 팀워크와 개인 기술이 조화를 이루어야 승리할 수 있다.')]

### 프롬프트 구성

In [32]:
template = """Answer the question based only on the following context:
{context}
 
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

### 모델 설정하기

In [33]:
# 구글 Gemini Pro 사용. 이 모델은 답변을 생성하는 모델이다.
# llm = HuggingFaceHub(repo_id = "google/flan-t5-xxl", model_kwargs={"temperature":0.8, "max_length": 512})
llm = ChatGoogleGenerativeAI(model="gemini-pro")

### 실행 체인 구성하기

In [34]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [36]:
chain.invoke('건강을 유지하려면 어떻게 해야하나요?')

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


'정기적인 운동과 균형 잡힌 식단'